In [1]:
import retworkx as rx
from retworkx.visualization import graphviz_draw


# Create the extended graph:
# connect every pair of nodes at distance below or equal than k.
def _build_distk_graph(graph, dist=None, k=1):
    if dist is None:
        dist = rx.graph_all_pairs_dijkstra_path_lengths(graph, lambda _: 1)

    nodes = graph.node_indexes()
    perm = {v: i for i, v in enumerate(nodes)}

    graph_k = rx.PyGraph(multigraph=False)
    graph_k.add_nodes_from(
        list(range(len(nodes)))
    )
    for v in nodes:
        for w, dval in dist[v].items():
            if dval <= k:
                graph_k.add_edge(perm[v], perm[w], None)
    
    return graph_k, perm



# Binary search over the "distance" threshold.
# This is equivalent to find the optimal value such that
# all "logical" qubits that interact in the circuit
# are mapped to physical qubits whose distance is below this value.
def bsearch(first, second):
    L = 1
    R = len(first) - 1
    
    dist = rx.graph_all_pairs_dijkstra_path_lengths(first, lambda _: 1)
    
    while L < R:
        mid = (L + R) // 2
        graph, _ = _build_distk_graph(first, dist, mid)
        res = rx.is_subgraph_isomorphic(graph, second,
                                        id_order=False, induced=False)
        
        if not res:
            L = mid + 1
        else:
            R = mid

    return L


# Compute the total cost := Sum over the cost of all edges in the interaction graph.
def cost(dist, im, mapping):
    tot = 0
    for u, v in im.edge_list():
        pu = mapping[u]
        pv = mapping[v]
        tot += (dist[pu][pv] - 1)

    return tot


# utility to visualize the coupling graph and the selected layout.
def visualize(cm, p2v):
    nodes = set(cm.node_indexes())
    maxn = max(nodes) + 1

    # "trick" to put weights in the original cm graph.
    # TODO: should make this easier in retworkx.
    graph = rx.PyGraph(multigraph=False)
    graph.add_nodes_from(list(range(maxn)))
    graph.remove_nodes_from([
        v for v in range(maxn) if v not in nodes
    ])
    
    graph.add_edges_from(
        list(cm.weighted_edge_list())
    )

    def node_attr(node):
        if node in p2v:
            return {
                'fillcolor': 'gray',
                'style': 'filled',
                'label': 'p' + str(node) + '-> q' + str(p2v[node])
            }

        return {'color': 'black', 'label': 'p' + str(node)}

    
    return graphviz_draw(graph, node_attr)

In [2]:
def cycle6_extra_edge():
    graph = rx.generators.cycle_graph(6)
    graph.add_edge(0, 3, None)
    
    return graph


testcases = {
    'ring_in_linear': {
        'cm': rx.generators.path_graph(15),
        'im': rx.generators.cycle_graph(5)
    },
    'star_in_hex': {
        'cm': rx.generators.hexagonal_lattice_graph(2, 1), 
        'im': rx.generators.star_graph(5)
    },
    'complete_in_hex': {
        'cm': rx.generators.hexagonal_lattice_graph(3, 3),
        'im': rx.generators.mesh_graph(6)
    },
    'high_degree': {
        'cm': rx.generators.grid_graph(3, 3), 
        'im': rx.generators.star_graph(6)
    },
    '6cycle_extra_edge_in_hex': {
        'cm': rx.generators.hexagonal_lattice_graph(2, 1),
        'im': cycle6_extra_edge(),
    },
    '6cycle_extra_edge_in_6cycle': {
        'cm': rx.generators.cycle_graph(6),
        'im': cycle6_extra_edge(),
    },
}

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
case = testcases['6cycle_extra_edge_in_hex']
cm = case['cm']
im = case['im']

opt_k = bsearch(cm, im)
print('Optimal distance threshold: ', opt_k)

Optimal distance threshold:  2


In [4]:
dist = rx.graph_all_pairs_dijkstra_path_lengths(cm, lambda _: 1)
cm_ext, perm = _build_distk_graph(cm, dist, opt_k)
perm = {new: old for old, new in perm.items()}

# find a mapping.
# this will return an iterator over all valid mappings.
# we just "store" the first one.
vf2 = rx.graph_vf2_mapping(cm_ext, im, subgraph=True,
                           id_order=False, induced=False)

try:
    p2v = next(vf2)
    p2v = {perm[k]: v for k, v in p2v.items()}
    # logical -> physical
    v2p = {v: k for k, v in p2v.items()}

    print('found mapping:\n', v2p)
    print('with cost:\n', cost(dist, im, v2p))
except StopIteration:
    print('no solution found. Why?')
    pass

found mapping:
 {3: 8, 0: 2, 5: 3, 1: 1, 4: 10, 2: 6}
with cost:
 4.0


In [5]:
visualize(cm, p2v)

RuntimeError: Graphviz could not be found or run. This function requires that Graphviz is installed. If you need to install Graphviz you can refer to: https://graphviz.org/download/#executable-packages for instructions.